In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vectors
from pyspark.sql.functions import udf
from pyspark.ml.linalg import VectorUDT, DenseVector
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType, StringType, ArrayType

import numpy as np

In [2]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
spark = SparkSession.builder.appName("SVM").getOrCreate()

23/12/01 10:54:29 WARN Utils: Your hostname, NN.local resolves to a loopback address: 127.0.0.1; using 192.168.0.191 instead (on interface en0)
23/12/01 10:54:29 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/01 10:54:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
data = spark.read.csv("../data/distilbert_imdb2.csv", header=True, inferSchema=True, sep=',')
data = data.withColumnRenamed("sentiment","label")

In [5]:
def parser(x):
    if x is None:
        return None
    elements = x.strip('[]').split(' ')
    result = [float(i) for i in elements if i.strip() != '']
    return (result) if result else None

parse_embedding_udf = udf(lambda x: parser(x), ArrayType(FloatType()))
data = data.withColumn("parsed_embeddings", parse_embedding_udf(data["embeddings_distilbert"]))

In [6]:
data = data.drop("embeddings_distilbert", "reviews_pre")

In [7]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

# UDF to convert array to Vector
vector_udf = udf(lambda a: Vectors.dense(a), VectorUDT())
data = data.withColumn("parsed_embeddings_vector", vector_udf(data["parsed_embeddings"]))

In [8]:
feature_cols = data.columns[1:-2] + data.columns[-1:]

assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")
data = assembler.transform(data)

In [9]:
train_data, test_data = data.randomSplit([0.7, 0.3], seed=42)

In [10]:
from pyspark.ml.classification import LinearSVC

svm = LinearSVC(labelCol="label", featuresCol="features")
svm_model = svm.fit(train_data)

Java HotSpot(TM) 64-Bit Server VM warning: CodeCache is full. Compiler has been disabled.
Java HotSpot(TM) 64-Bit Server VM warning: Try increasing the code cache size using -XX:ReservedCodeCacheSize=


CodeCache: size=131072Kb used=26206Kb max_used=26507Kb free=104865Kb
 bounds [0x00000001089d8000, 0x000000010a3d8000, 0x00000001109d8000]
 total_blobs=10157 nmethods=9192 adapters=876
 compilation: disabled (not enough contiguous free space left)


23/12/01 10:55:12 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


In [11]:
predictions = svm_model.transform(test_data)

In [13]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

evaluator = BinaryClassificationEvaluator()
auc = evaluator.evaluate(predictions)
print(f"AUC: {auc}")

AUC: 0.9113693440900237


In [14]:
model_path = "../models/pyspark_distilbert_SVM_model"
svm_model.save(model_path)

In [15]:
spark.stop()